In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Analise de satisfação de passageiros em companhias aereas 

***

# Conteúdo 

### 0 [Libraries](#section0)

### 1 [Informação e obejtivo da Analise do Base de Dados](#section1)

### 2 [Analise Exploratoria](#section2)

### 3 [Tratamento das bases treino e teste](#section3)

### 4 [Random Forest Classifier](#section4)

### 5 [Concluões](#section7)

***

# 0 Libraries <a class="anchor" id="section0"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2
from sklearn.preprocessing import LabelEncoder, StandardScaler


# 3
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import scikitplot as skplt




***

# 1 Informação e obejtivo da Analise do Base de Dados  <a class="anchor" id="section1"></a>

Esse banco de dados contém informações de uma pesquisa de satisfação de passageiros de algumas  companhias aéreas. O objetivo neste trabalho e utilizar e execução modelo de ML usando sckit-learn, encontrar padrões que afetam a decisção dos clientes em relação a qualidades do serviço prestado por essas companhias e no final ter um algoritmos  prever se o cliente está ou não satisfeito com o serviço oferecido.

**Dicionário:**
1. Gender: Gênero do Passageiro (Female, Male)
2. Customer Type: Tipo do cliente (Loyal customer, disloyal customer)
3. Age: Idade dos passageiros 
4. Type of Travel: Finalidade da viagem do passageiro (Personal Travel, Business Travel)
5. Class: Classe de viagem que o passageiro está no voo (Business, Eco, Eco Plus)
6. Flight distance: Distância da viagem que o passageiro realizou
7. Inflight wifi service: Serviço de wifi a bordo (0:Not Applicable; 1-5)
8. Departure/Arrival time convenient: Satisfação no tempo de partida;chegada
9. Ease of Online booking: Satisfação de reserva online
10. Gate location: Satisfação de localização do porta 
11. Food and drink: Satisfação com comida e bebida
12. Online boarding: Satisfaction level of online boarding
13. Seat comfort: Satisfaction level of seat comfort
14. Inflight entertainment: Satisfaction level of inflight entertainment
15. On-board service: Satisfaction level of on-board service
16. Leg room service: Satisfaction level of leg room service
17. Baggage handling: Satisfaction level of baggage handling
18. Check-in service: Satisfaction level of check-in service
19. Inflight service: Satisfaction level of inflight service
20. Cleanliness: Satisfaction level of Cleanliness
21. Departure Delay in Minutes: Minutes delayed when departure
22. Arrival Delay in Minutes: Minutes delayed when Arrival
23. Satisfaction: Airline satisfaction level (Satisfaction, neutral or dissatisfaction)

In [ ]:
# Carregando as bases de treino e teste:
treino = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
teste = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/test.csv')

treino.shape, teste.shape

***

# 2 Analise Exploratoria dos dados <a class="anchor" id="section2"></a>

In [ ]:
df = treino.append(teste)

In [ ]:
df.head(2).T

In [ ]:
#The columns `Unnamed: 0` and `id` are irrelevant and thus will be dropped.

df.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.describe().transpose()

In [ ]:
hist, ax=plt.subplots()
ax = sns.distplot(df['Arrival Delay in Minutes'], rug=False)
ax.set_xlabel('Atraso na chegada (min)')
#ax.set_ylabel('Frenquency')
plt.show()

In [ ]:
df1 = df[df['Arrival Delay in Minutes'] <= 50]

In [ ]:
df1.shape

In [ ]:
hist, ax=plt.subplots()
ax = sns.distplot(df1['Arrival Delay in Minutes'], rug=False)
ax.set_xlabel('Atraso na chegada (min)')
#ax.set_ylabel('Frenquency')
plt.show()

In [ ]:
df_sem_na = df.fillna(df.median())

df_sem_na.shape

In [ ]:
df_sem_na.isna().sum()

In [ ]:
sns.set_theme(style='ticks', palette='Paired')
sns.set_context("paper")

### Analisando a Variavél Target - 'Satisfaction'

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='satisfaction', data=df_sem_na)

for a in ax.patches:
    ax.annotate(np.round(float((a.get_height()/129880)*100),decimals=2),
            (a.get_x()+a.get_width()/2, a.get_height()),
            ha='center',
            va='center',
            xytext=(0, 10),
            textcoords='offset points',
            fontsize = 10,
          ),
            

plt.title('Gráfico de Satisfação (Target)', fontsize=14)
plt.xlabel('Satisfação (Target)', fontsize=13)
plt.ylabel('Count', fontsize=13)
plt.show()

#### Obs 01.: Os dados bem  equilibrados, com 56.55 % dos clientes neutros ou insatisfeitos e 43.45 % satisfeitos.

### Analisando as Variavéis Categoricas (String)

In [ ]:
# criando lista com um determinado grupo de variavies categoricas (string)
categ_cols = df_sem_na[['Gender', 'Customer Type', 'Type of Travel', 'Class']]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))
axs = axs.ravel()

i=0
for col in categ_cols:
    
    # define location
    plt.subplot(2, 2, i+1)
    
    # create plot
    ax = plt.gca()
    axs[i] = sns.countplot(x = col, hue = 'satisfaction', data = df)
    
   
    i += 1    

plt.tight_layout()
plt.show()

Obs. 02:
- A variavel genero (Gender) tem informações de satisfação bem semelhantes, tendo uma leve discrepancia no genero feminino com insastisfação maior;
- Na variavél tipo de cliente temos que pessoas sem fidelidade são mais insatisfeitas;
- Pessoas que não viajam a passeio são mais insatisfeitas;
- Como de se esperar pessoas da classe economica são mais insatisfeita, inversamente as pessoas da classe executiva que estão satisfeitas durante todo o processo do voo.

In [ ]:
# criando  um determinado grupo de variavies categoricas 
categ1_cols = df_sem_na[['Inflight wifi service','Departure/Arrival time convenient','Ease of Online booking','Gate location','Food and drink','Online boarding','Seat comfort','Inflight entertainment','On-board service','Leg room service','Baggage handling','Checkin service','Inflight service','Cleanliness']]

In [ ]:
df2 = df_sem_na[df_sem_na['Class'] == 'Business']

In [ ]:
df3 = df_sem_na[df_sem_na['Class'] == 'Eco']

In [ ]:
fig, axs = plt.subplots(nrows=5, ncols=3, figsize=(14, 10))
axs = axs.ravel()

i=0
for col in categ1_cols:
    
    # define location
    plt.subplot(5, 3, i+1)
    
    # create plot
    ax = plt.gca()
    axs[i] = sns.countplot(x = col, hue = 'satisfaction', data = df_sem_na)
    
   
    i += 1    

plt.tight_layout()
plt.show()

Obs. 03:

- Da para perceber que dependendo do perfil do cliente algumas variaveis podem pesar na decições de satisfação mais do que outras, assim também a combinaçao entre variaveís. Por exemplo quando olhamos separadamente os viajantes da classe Economica/Executiva, variéaveis influenciam diferentemente ou variáveis que não fazem tanta diferença que o resultado é mais equlibrado. O importante é que a base ´tem padrões nitidos que pode dar uma boa apredizagem de maquina para o nosso modelo.

### Analise de Correlação

In [ ]:
# criar  labels para as : variáveis 'Gender', 'Customer Type', 'Type of Travel', 'Class'
label = LabelEncoder()

labeled_df = df_sem_na.copy()
for i in categ_cols:
    labeled_df[i] = label.fit_transform(labeled_df[i])

In [ ]:
cluster = labeled_df.corr(method = 'pearson')
mask = np.zeros_like(cluster, dtype = np.bool)

mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(220,10, as_cmap = True)

In [ ]:
fig, ax=plt.subplots(figsize=(12,6))
sns.heatmap(cluster, annot=True, linewidths=0.5,ax=ax, fmt='.1f',cmap=cmap)
fig.suptitle('Mapa de Calor ')

In [ ]:
cor = df_sem_na['Departure Delay in Minutes'].corr(df_sem_na['Arrival Delay in Minutes'])

In [ ]:
cor

Obs 04.:

Correlação das variáveis: `Departure Delay in Minutes` e `Arrival Delay in Minutes` tem uma correlação de:  0.96. Com isso podemos resolver o problema dos nulos da Arrival Delay in Minutes` e retirar da base para teino!

Outras variaveis que também se correlacionaram bem:

- 'Ease of Online booking' e 'Inflight wifi service'
- 'cleanlises' com 'Food and Drink', 'Seat confort' e 'Inflight entertaiment'

# 3 Tratamento das bases treino/teste <a class="anchor" id="section3"></a>

In [ ]:
# Retirando a variavel `Arrival Delay in Minutes`, devido ter valores zeros. Como temmuita correlação com a variavel `Departure Delay in Minutes`
treino.drop('Arrival Delay in Minutes',axis = 1,inplace=True)
teste.drop('Arrival Delay in Minutes',axis = 1,inplace=True)

In [ ]:
def transform_satisfaction(x):
    if x == 'satisfied':
        return 1
    elif x == 'neutral or dissatisfied':
        return 0
    else:
        return -1

In [ ]:
# transformando a variavel target de strig para númerico 
treino['satisfaction'] = treino['satisfaction'].apply(transform_satisfaction)
teste['satisfaction'] = teste['satisfaction'].apply(transform_satisfaction)

In [ ]:
dummy_treino = pd.get_dummies(treino, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'])
dummy_teste = pd.get_dummies(teste, columns=['Gender', 'Customer Type', 'Type of Travel', 'Class'])

In [ ]:
treino_df = dummy_treino.copy()
teste_df = dummy_teste.copy()

### Escalomamento

padronização de dados é o processo de redimensionar os atributos para que tenham média igual a 0 e variância igual a 1.O objetivo final de realizar a padronização é reduzir todos os recursos a uma escala comum, sem distorcer as diferenças na faixa dos valores.

In [ ]:
treino_cols = treino_df[['Age', 'Flight Distance', 'Departure Delay in Minutes']]
teste_cols = teste_df[['Age', 'Flight Distance', 'Departure Delay in Minutes']]

In [ ]:
scaler = StandardScaler()
scaled_treino = pd.DataFrame(scaler.fit_transform(treino_cols), columns = treino_cols.columns)
scaled_teste = pd.DataFrame(scaler.fit_transform(teste_cols), columns = teste_cols.columns)

In [ ]:
scaled_treino.head().style.set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

In [ ]:
cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes']

treino_df[cols] = scaled_treino
teste_df[cols] = scaled_teste

In [ ]:
treino_df.head().style.set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

In [ ]:
df_rf = treino_df.append(teste_df)

***

# 4  Random Forest Classifier <a class="anchor" id="section4.3"></a>

In [ ]:
# Separando o dataframe em train, valid e test

# Primeiro, train e test
train, test = train_test_split(df_rf, test_size=0.2, random_state=42)

# Depois, train e valid
train, valid = train_test_split(treino_df, test_size=0.2, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# Colunas a serem usadas para treino
feat = [c for c in df_rf.columns if c not in ['Unnamed: 0','id', 'satisfaction']]

In [ ]:
# Instanciando o modelo
rf = RandomForestClassifier(n_estimators=100, random_state=10)

# Treinando o modelo
rf.fit(train[feat], train['satisfaction'])

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=feat).sort_values().plot.barh()

In [ ]:
# Previsões para os dados de teste
preds_test = rf.predict(test[feat])

# Verificando a acurácia
accuracy_score(test['satisfaction'], preds_test)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(test['satisfaction'], preds_test)

In [ ]:
# Prevendo os dados de validação
preds_val = rf.predict(valid[feat])

# Verificando a acurácia
accuracy_score(valid['satisfaction'], preds_val)

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(valid['satisfaction'], preds_val)

***

# 5 Conclusões <a class="anchor" id="section7"></a>

Neste trabalho o principal objetivo era encontrar as caracteristicas que mais pesavam na satisfação dos clientes dentre as variaveis categoricas. Também verificamos a qualidade do modelo com o calculo da acuracia (teste 0,98 e validado 0,96) e da matrix de confunsão para os dados de  teste e validados. 

As variáveis categoricas com maior relevancia para a escolha do cliente está satisfeito foram:

- Inflight entertainment
- Online boarding
- Inflight wifi service
- Type of Travel 

***